## Problem

Input : 
- จุดเริ่มต้น : ในที่นี้ให้เป็น A
- งาน(ต้นทาง-ปลายทาง) + จำนวนของแต่ละงาน : จากข้างล่างคือ table tasks
- ระยะทางระหว่างจุดทั้งหมด : ตัวแปร distances

Output : 
- หาทุกทางที่ทำให้งานจบ
- ทุกทางยาวเท่าไหร่
- บอกด้วยว่าจากระยะทางทั้งหมด แบ่งเป็นระยะทางมีของและไม่มีของอย่างละเท่าไหร่
- เอาทางที่ระยะทางซ้ำกันออก

Import Library

In [1]:
import numpy as np
import pandas as pd

### Input

In [34]:
start_point = np.array([['A']])
distances = np.load('data/distances.npy')
tasks = pd.read_csv('data/tasks.csv')
tasks['path'] = tasks['start'] +'-'+ tasks['end']
node_names = np.load('data/node_name.npy')

In [35]:
start_point

array([['A']], dtype='<U1')

In [36]:
distances

array([[  0,  65,   7,  46, 284],
       [ 65,   0,  88,  67,  86],
       [  7,  88,   0, 121, 294],
       [ 46,  67, 121,   0, 194],
       [284,  86, 294, 194,   0]])

In [37]:
tasks

,counts,start,end,path
0,2,A,B,A-B
1,1,A,C,A-C
2,1,A,E,A-E
3,1,C,B,C-B
4,2,C,D,C-D


In [38]:
node_names # ชื่อของ node ทั้งหมด เอาไว้ map กับ matrix distance

array(['A', 'B', 'C', 'D', 'E'], dtype='<U1')

# Solve

#### Idea
- ถ้าอยู่จุดรับ(มีของ) คิดว่ามีงานไรที่เริ่มจากจุดเราบ้าง (จากตาราง tasks ถ้าเราอยู่จุด A เราจะไปได้ 3 จุดคือ B, C, E)
- อยู่จุดส่ง(ว่าง) มองหาจุดรับ (ถ้าอยู่ที่ B ก็ต้องกลับไปไม่ A ก็ C เพื่อรับของไปส่งใหม่)
- ทำซ้ำไปเรื่อยๆ จนงานหมด

จาก idea พบว่าน่าจะมี function สำหรับแต่ละงานดังนี้
1. จาก table tasks ถ้าใส่ path ปัจจุบันเข้าไปแล้วบอกว่าตอนนี้เหลือ task อะไรบ้าง เช่น [A-B,B-A,A-C,C-X] อาจบอกว่าเหลือ 2 task คือ A-E กับ C-D
2. กรณีที่งานยังไม่หมด check ว่าที่ตัวเองอยู่เป็นจุดรับหรือส่ง แล้วบอกว่าจะไปจุดไหนต่อจากจุดนี้
    - ถ้าอยู่ส่ง ควรไปจุดรับ
    - ถ้าอยู่จุดรับ(หรือจุดที่ไม่ใช่จุดส่ง) ควรกลับไปจุดส่ง
3. ฟังก์ชั่นสร้าง matrix ของงานทั้งหมดที่เป็นไปได้ใน step ถัดไป จาก งานที่ทำไปแล้ว(row หนึ่ง) และ task ที่เหลือ(เอา row ยัดใส่ 1.)
4. ทำซ้ำเรื่อยๆ จน column สุดท้ายเป็น '0' หมด
5. คำนวณระยะทางของแต่ละเส้นทางจากตัวแปร distance
    - แบ่งเป็นระยะทางที่ + และ - 
    - ลบ path ที่ให้ผลเหมือนกันออก

In [130]:
# 1. for each path - check how much the job done.
def checkRemainingTasks(job_done, tasks):
    remaining_tasks = tasks.copy()
    active_path = tasks['path'].values
    
    # สนเฉพาะ job ที่อยู่ใน tasks
    job_done = job_done[np.isin(job_done, active_path)] 
    
    for job in job_done:
        if len(remaining_tasks) > 0:
            remaining_tasks.loc[remaining_tasks['path'] == job, 'counts'] = remaining_tasks[remaining_tasks['path'] == job]['counts'] - 1
            remaining_tasks = remaining_tasks[remaining_tasks['counts'] != 0]

            number_of_rmtasks = sum(remaining_tasks['counts'])
            
    # return task ที่ job ยังไม่ได้ทำ ถ้าทำหมดแล้วจะ return table เปล่า
    return remaining_tasks

# 2.
def startOrEnd(current_position,rm_task):
    if len(rm_task) > 0:
        start_set = rm_tasks['start'].unique()
        end_set = rm_tasks['end'].unique()
    
        # ถ้าอยู่ในกลุ่ม start ต้องไปจุด end ของ position ที่เราอยู่
        if current_position in start_set:
            return rm_task[rm_task['start'] == current_position]['end'].values
        # ถ้าอยู่ในกลุ่ม end ต้องไปกลุ่ม start
        elif current_position in end_set:
            return start_set
        
        # ถ้าไม่อยู่ทั้งคู่ไปจุด start
        else:
            return start_set
    else:
        return 0

# 3.
def nextJobsMatrix(job_done,next_nodes):
    current_position = job_done[-1].split('-')[-1]
    next_jobs = current_position + '-' + next_nodes
    
    output = np.empty((0,len(job_done) + 1), str) if len(job_done[0]) > 1 else np.empty((0,1), str)

    for next_job in next_jobs:
        if len(job_done[0]) > 1:
            next_job_done = np.append(job_done,next_job)
            output = np.vstack((output, next_job_done))
        else:
            output = np.vstack((output, next_job))
    return output

# 4.
def allTasksPath(before_path,tasks):
    output = np.empty((0,before_path.shape[1] + 1), str) if len(before_path[0,0]) > 1 else np.empty((0,1), str)
    
    for each_row in before_path:
        current_position = each_row[-1].split('-')[-1]
        remaining_jobs = checkRemainingTasks(each_row,tasks)

        if len(remaining_jobs) == 0:
            job_matrix = np.append(each_row,'0')
        else:
            next_nodes = startOrEnd(current_position,remaining_jobs)
            job_matrix = nextJobsMatrix(each_row,next_nodes)
        output = np.vstack((output, job_matrix))

    last_column = output[:,-1]
    all_zero = sum(last_column == '0')
    print('Output :')
    print(output)
    if all_zero == len(last_column):
        return output
    else:
        return allTasksPath(output,tasks)

#### ทดสอบ

ตอนอยู่ A ที่จุดแรกเลย

In [112]:
start_point

array([['A']], dtype='<U1')

In [113]:
current_position = start_point[0][-1].split('-')[-1]
current_position

'A'

In [114]:
rm_tasks = checkRemainingTasks(start_point,tasks)
rm_tasks

,counts,start,end,path
0,2,A,B,A-B
1,1,A,C,A-C
2,1,A,E,A-E
3,1,C,B,C-B
4,2,C,D,C-D


อยู่ที่จุดเริ่ม งานที่เหลือเท่ากับงานทั้งหมด

In [115]:
next_nodes = startOrEnd(current_position,rm_tasks)
next_nodes

array(['B', 'C', 'E'], dtype=object)

จากจุด A ผลออกมาว่าควรไป B,C,E 

In [116]:
next_jobs = current_position + '-' + next_nodes
next_jobs

array(['A-B', 'A-C', 'A-E'], dtype=object)

ทำให้อยู่ใน format Start-End

ลองทำเป็น recursive function stack matrix ซ้อนกันเรื่อยๆ จะได้

In [131]:
all_paths = allTasksPath(start_point,tasks)

Output :
[['A-B']
 ['A-C']
 ['A-E']]
Output :
[['A-B' 'B-A']
 ['A-B' 'B-C']
 ['A-C' 'C-B']
 ['A-C' 'C-D']
 ['A-E' 'E-A']
 ['A-E' 'E-C']]
Output :
[['A-B' 'B-A' 'A-B']
 ['A-B' 'B-A' 'A-C']
 ['A-B' 'B-A' 'A-E']
 ['A-B' 'B-C' 'C-B']
 ['A-B' 'B-C' 'C-D']
 ['A-C' 'C-B' 'B-A']
 ['A-C' 'C-B' 'B-C']
 ['A-C' 'C-D' 'D-A']
 ['A-C' 'C-D' 'D-C']
 ['A-E' 'E-A' 'A-B']
 ['A-E' 'E-A' 'A-C']
 ['A-E' 'E-C' 'C-B']
 ['A-E' 'E-C' 'C-D']]
Output :
[['A-B' 'B-A' 'A-B' 'B-A']
 ['A-B' 'B-A' 'A-B' 'B-C']
 ['A-B' 'B-A' 'A-C' 'C-B']
 ['A-B' 'B-A' 'A-C' 'C-D']
 ['A-B' 'B-A' 'A-E' 'E-A']
 ['A-B' 'B-A' 'A-E' 'E-C']
 ['A-B' 'B-C' 'C-B' 'B-A']
 ['A-B' 'B-C' 'C-B' 'B-C']
 ['A-B' 'B-C' 'C-D' 'D-A']
 ['A-B' 'B-C' 'C-D' 'D-C']
 ['A-C' 'C-B' 'B-A' 'A-B']
 ['A-C' 'C-B' 'B-A' 'A-E']
 ['A-C' 'C-B' 'B-C' 'C-D']
 ['A-C' 'C-D' 'D-A' 'A-B']
 ['A-C' 'C-D' 'D-A' 'A-E']
 ['A-C' 'C-D' 'D-C' 'C-B']
 ['A-C' 'C-D' 'D-C' 'C-D']
 ['A-E' 'E-A' 'A-B' 'B-A']
 ['A-E' 'E-A' 'A-B' 'B-C']
 ['A-E' 'E-A' 'A-C' 'C-B']
 ['A-E' 'E-A' 'A-C' 'C-D']
 ['A

ลองเชคคำตอบ

In [135]:
all_paths.shape

(450, 14)

In [134]:
all_paths

array([['A-B', 'B-A', 'A-B', ..., 'C-D', '0', '0'],
       ['A-B', 'B-A', 'A-B', ..., 'C-D', '0', '0'],
       ['A-B', 'B-A', 'A-B', ..., 'A-E', '0', '0'],
       ...,
       ['A-E', 'E-C', 'C-D', ..., 'B-A', 'A-C', '0'],
       ['A-E', 'E-C', 'C-D', ..., 'A-B', '0', '0'],
       ['A-E', 'E-C', 'C-D', ..., 'B-A', 'A-C', '0']], dtype='<U3')

จะเห็นว่า columns สุดท้ายเป็น 0 หมดแล้ว

In [145]:
all_paths[309]

array(['A-C', 'C-D', 'D-C', 'C-B', 'B-A', 'A-B', 'B-A', 'A-E', 'E-C',
       'C-D', 'D-A', 'A-B', '0', '0'], dtype='<U3')

สุ่มทดสองสัก tasks ดูว่างานหมดจริงไหม

In [146]:
checkRemainingTasks(all_paths[309], tasks)

,counts,start,end,path


งานหมดจริง

คำนวณ distances

In [147]:
distances

array([[  0,  65,   7,  46, 284],
       [ 65,   0,  88,  67,  86],
       [  7,  88,   0, 121, 294],
       [ 46,  67, 121,   0, 194],
       [284,  86, 294, 194,   0]])